In [1]:
pip install geoip2 pandas 


  Using cached geoip2-4.8.1-py3-none-any.whl.metadata (18 kB)
  Using cached aiohttp-3.11.8-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached maxminddb-2.6.2-cp312-cp312-win_amd64.whl.metadata (5.2 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.0-cp312-cp312-win_amd64.whl.metadata (7.9 kB)
  Using cached yarl-1.18.0-cp312-cp312-win_amd64.whl.metadata (69 kB)
Using cached geoip2-4.8.1-py3-none-any.whl (27 kB)
Using cached aiohttp-3.11.8-cp312-cp312-win_amd64.whl (437 kB)
Using cached maxminddb-2.6.2-cp312-cp312-win_amd64.whl (36 kB)
Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl (14 kB)
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl (51 kB

In [2]:
import pandas as pd

# Загрузка датасета
file_path = r"dataset.csv"  # Укажите путь к вашему файлу
data = pd.read_csv(file_path)

# Проверка структуры данных
print(data.head())

# Проверка на пропущенные значения
print(data.isnull().sum())

   Unnamed: 0  transaction_id             ip  device_id device_type  \
0           0         1157921    91.107.0.90      982.0   prtbl trm   
1           1         1595205   37.0.127.150     1033.0    port_trm   
2           2         1656884  185.233.0.240       96.0         atm   
3           3         1673909    91.107.0.90      982.0   prtbl trm   
4           4         1689203   188.43.0.135     1413.0         ATM   

   tran_code   mcc  client_id card_type  pin_inc_count card_status  \
0         30  7103  969838140     DEBIT              0      active   
1         30  2031  969838140     DEBIT              0      active   
2         25  4629  969838140     DEBIT              0      active   
3         30  4629  969838140     DEBIT              0      active   
4         10  5840  969838140     DEBIT              0      active   

  expiration_date             datetime     sum   oper_type    balance  
0      2030-06-10  2022-03-09 09:56:51  510.22    transfer  670814.78  
1      2

In [5]:
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
# Удаляем столбец 'Unnamed: 0'
data = data.drop(columns=['Unnamed: 0'])

# Сортировка данных по 'client_id' datetime
#data_sorted = data.sort_values(by=['client_id', 'datetime'], ascending=[True, True]).reset_index(drop=True)
data_sorted = data

# Печать отсортированного датасета
print(data_sorted)

       transaction_id             ip  device_id device_type  tran_code   mcc  \
0             1157921    91.107.0.90      982.0   prtbl trm         30  7103   
1             1595205   37.0.127.150     1033.0    port_trm         30  2031   
2             1656884  185.233.0.240       96.0         atm         25  4629   
3             1673909    91.107.0.90      982.0   prtbl trm         30  4629   
4             1689203   188.43.0.135     1413.0         ATM         10  5840   
...               ...            ...        ...         ...        ...   ...   
53074         9919604    91.122.0.15      431.0         atm         25  3193   
53075         9902369    91.122.0.15      431.0         atm         10  7071   
53076         9939711     2.94.0.240     1042.0         atm         10  3686   
53077         9876313   188.0.162.75      494.0         atm         25  7602   
53078         9791767    46.0.43.105     1489.0         atm         20  9729   

       client_id card_type  pin_inc_cou

In [7]:
import geoip2.database

# Путь к базе данных GeoLite2 (если файл в той же папке)
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

# Функция для получения города по IP-адресу
def get_city_from_ip(ip):
    try:
        # Получаем данные о местоположении
        response = reader.city(ip)
        city = response.city.name  # Извлекаем название города
        
        # Если город не найден (None), возвращаем 'Unknown'
        if city is None:
            return 'Unknown'
        
        return city
    except geoip2.errors.AddressNotFoundError:
        return 'Unknown'  # Если IP-адрес не найден

# Применение функции к вашему датасету
data_sorted['ip_city'] = data_sorted['ip'].apply(get_city_from_ip)

# Посмотрим на результат
data_sorted[['ip', 'ip_city']].head()


,ip,ip_city
0,91.107.0.90,Tehran
1,37.0.127.150,Moscow
2,185.233.0.240,Almaty
3,91.107.0.90,Tehran
4,188.43.0.135,Unknown


In [4]:
# Проверяем уникальные значения и их количество для каждого столбца
device_type_values = data_sorted['device_type'].value_counts()
card_type_values = data_sorted['card_type'].value_counts()
card_status_values = data_sorted['card_status'].value_counts()
oper_type_values = data_sorted['oper_type'].value_counts()
ip_city_values = data_sorted['ip_city'].value_counts()

# Печатаем результаты
print("Device Type Values:")
print(device_type_values)
print("\nCard Type Values:")
print(card_type_values)
print("\nCard Status Values:")
print(card_status_values)
print("\nOper Type Values:")
print(oper_type_values)
print("\nOper Type Values:")
print(ip_city_values)

Device Type Values:
device_type
atm              10592
ATM               9975
cash_in           7555
cash_out          7306
pos trm           4985
prtbl trm         4317
port_trm          4315
Portable term     4034
Name: count, dtype: int64

Card Type Values:
card_type
CREDIT    27091
DEBIT     25988
Name: count, dtype: int64

Card Status Values:
card_status
active     27451
act        25539
blk           48
blocked       41
Name: count, dtype: int64

Oper Type Values:
oper_type
payment             13204
transfer             8826
diff_cntry           4204
country_transfer     4155
in_acc               2769
decrease_on_acc      2665
in                   2657
from_acc             2595
out                  2587
add_on_acc           2515
bad                  2350
err_code             2312
err                  2238
blocked                 1
blk                     1
Name: count, dtype: int64

Oper Type Values:
ip_city
Unknown                     16820
Samara                       9388
Mosc

In [8]:
# Приводим к общим признакам для device_type
device_type_mapping = {
    'ATM': 'atm',
    'atm': 'atm',  # Если хотите оставить 'atm' как общий
    'cash_in': 'cash_in',
    'cash_out': 'cash_out',
    'pos trm': 'pos_trm',
    'prtbl trm': 'prtbl_trm',
    'port_trm': 'prtbl_trm',
    'Portable term': 'prtbl_trm'  # Можно объединить с port_trm
}

data_sorted['device_type'] = data_sorted['device_type'].replace(device_type_mapping)

# Приводим к общим признакам для card_status
card_status_mapping = {
    'active': 'active',
    'act': 'active',  # Объединяем 'act' с 'active'
    'blk': 'blocked',  # Объединяем 'blk' и 'blocked'
    'blocked': 'blocked'
}

data_sorted['card_status'] = data_sorted['card_status'].replace(card_status_mapping)

# Приводим к общим признакам для oper_type
oper_type_mapping = {
    'payment': 'payment',
    'transfer': 'transfer',
    'diff_cntry': 'transfer',  # Приводим к одному типу операции
    'country_transfer': 'transfer',
    'in_acc': 'in_acc',
    'decrease_on_acc': 'decrease_on_acc',
    'in': 'in_acc',
    'from_acc': 'from_acc',
    'out': 'from_acc',
    'add_on_acc': 'add_on_acc',
    'bad': 'bad',
    'err_code': 'error',
    'err': 'error',
    'blocked': 'blocked',
    'blk': 'blocked'  # Приводим 'blk' к 'blocked'
}

data_sorted['oper_type'] = data_sorted['oper_type'].replace(oper_type_mapping)

# Просмотр обновленных значений
print("Updated Device Type Values:")
print(data_sorted['device_type'].value_counts())

print("\nUpdated Card Status Values:")                            
print(data_sorted['card_status'].value_counts())

print("\nUpdated Oper Type Values:")
print(data_sorted['oper_type'].value_counts())



Updated Device Type Values:
device_type
atm          20567
prtbl_trm    12666
cash_in       7555
cash_out      7306
pos_trm       4985
Name: count, dtype: int64

Updated Card Status Values:
card_status
active     52990
blocked       89
Name: count, dtype: int64

Updated Oper Type Values:
oper_type
transfer           17185
payment            13204
in_acc              5426
from_acc            5182
error               4550
decrease_on_acc     2665
add_on_acc          2515
bad                 2350
blocked                2
Name: count, dtype: int64


In [9]:
# Сохранение измененного датасета в CSV
data_sorted.to_csv('modified_data.csv', index=False)

# Проверим, что файл был сохранен
print("Датасет успешно сохранен в файл 'modified_data.csv'")

Датасет успешно сохранен в файл 'modified_data.csv'


In [10]:
# Печать отсортированного датасета
print(data_sorted)

       transaction_id             ip  device_id device_type  tran_code   mcc  \
0             1157921    91.107.0.90      982.0   prtbl_trm         30  7103   
1             1595205   37.0.127.150     1033.0   prtbl_trm         30  2031   
2             1656884  185.233.0.240       96.0         atm         25  4629   
3             1673909    91.107.0.90      982.0   prtbl_trm         30  4629   
4             1689203   188.43.0.135     1413.0         atm         10  5840   
...               ...            ...        ...         ...        ...   ...   
53074         9919604    91.122.0.15      431.0         atm         25  3193   
53075         9902369    91.122.0.15      431.0         atm         10  7071   
53076         9939711     2.94.0.240     1042.0         atm         10  3686   
53077         9876313   188.0.162.75      494.0         atm         25  7602   
53078         9791767    46.0.43.105     1489.0         atm         20  9729   

       client_id card_type  pin_inc_cou

In [11]:
# Загрузка датасета
file_path2 = r"modified_data.csv"  # Укажите путь к вашему файлу
data2 = pd.read_csv(file_path2)

# Проверка структуры данных
print(data2.head())

   transaction_id             ip  device_id device_type  tran_code   mcc  \
0         1157921    91.107.0.90      982.0   prtbl_trm         30  7103   
1         1595205   37.0.127.150     1033.0   prtbl_trm         30  2031   
2         1656884  185.233.0.240       96.0         atm         25  4629   
3         1673909    91.107.0.90      982.0   prtbl_trm         30  4629   
4         1689203   188.43.0.135     1413.0         atm         10  5840   

   client_id card_type  pin_inc_count card_status expiration_date  \
0  969838140     DEBIT              0      active      2030-06-10   
1  969838140     DEBIT              0      active      2030-06-10   
2  969838140     DEBIT              0      active      2030-06-10   
3  969838140     DEBIT              0      active      2030-06-10   
4  969838140     DEBIT              0      active      2030-06-10   

              datetime     sum oper_type    balance  ip_city  
0  2022-03-09 09:56:51  510.22  transfer  670814.78   Tehran  
1 

In [12]:
from sklearn.preprocessing import LabelEncoder

# Создание объекта LabelEncoder
label_encoder = LabelEncoder()

# Список столбцов, для которых нужно применить Label Encoding
columns_to_encode = ['device_type', 'card_type', 'card_status', 'oper_type', 'ip_city']

# Применяем Label Encoding к каждому столбцу
for column in columns_to_encode:
    data_sorted[column] = label_encoder.fit_transform(data_sorted[column])

# Проверим результат (первые несколько строк)
print(data_sorted.head())



   transaction_id             ip  device_id  device_type  tran_code   mcc  \
0         1157921    91.107.0.90      982.0            4         30  7103   
1         1595205   37.0.127.150     1033.0            4         30  2031   
2         1656884  185.233.0.240       96.0            0         25  4629   
3         1673909    91.107.0.90      982.0            4         30  4629   
4         1689203   188.43.0.135     1413.0            0         10  5840   

   client_id  card_type  pin_inc_count  card_status expiration_date  \
0  969838140          1              0            0      2030-06-10   
1  969838140          1              0            0      2030-06-10   
2  969838140          1              0            0      2030-06-10   
3  969838140          1              0            0      2030-06-10   
4  969838140          1              0            0      2030-06-10   

             datetime     sum  oper_type    balance  ip_city  
0 2022-03-09 09:56:51  510.22          8  67081

In [13]:
# Проверяем уникальные значения и их количество для каждого столбца
device_type_values = data_sorted['device_type'].value_counts()
card_type_values = data_sorted['card_type'].value_counts()
card_status_values = data_sorted['card_status'].value_counts()
oper_type_values = data_sorted['oper_type'].value_counts()
ip_city_values = data_sorted['ip_city'].value_counts()

# Печатаем результаты
print("Device Type Values:")
print(device_type_values)
print("\nCard Type Values:")
print(card_type_values)
print("\nCard Status Values:")
print(card_status_values)
print("\nOper Type Values:")
print(oper_type_values)
print("\nOper Type Values:")
print(ip_city_values)

Device Type Values:
device_type
0    20567
4    12666
1     7555
2     7306
3     4985
Name: count, dtype: int64

Card Type Values:
card_type
0    27091
1    25988
Name: count, dtype: int64

Card Status Values:
card_status
0    52990
1       89
Name: count, dtype: int64

Oper Type Values:
oper_type
8    17185
7    13204
6     5426
5     5182
4     4550
3     2665
0     2515
1     2350
2        2
Name: count, dtype: int64

Oper Type Values:
ip_city
24    16820
18     9388
11     7261
5      2527
26     1671
13     1550
17      928
4       924
23      915
25      897
20      896
12      856
9       855
6       847
0       843
16      837
10      811
15      782
22      772
14      726
3       666
1       624
27      361
2       104
7        44
19       44
28       44
8        43
21       43
Name: count, dtype: int64


In [14]:
# Сохранение измененного датасета в CSV
data_sorted.to_csv('modified_data_encode.csv', index=False)

# Проверим, что файл был сохранен
print("Датасет успешно сохранен в файл 'modified_data_encode.csv'")

Датасет успешно сохранен в файл 'modified_data_encode.csv'


In [15]:
# Загрузка датасета
file_path3 = r"modified_data_encode.csv"  # Укажите путь к вашему файлу
data3 = pd.read_csv(file_path3)# Сортировка данных по 'client_id' datetime
#data_sorted = data3.sort_values(by=['client_id', 'datetime'], ascending=[True, True]).reset_index(drop=True)


# Печать отсортированного датасета
print(data_sorted)
data_sorted.to_csv('modified_data_encode_sorted.csv', index=False)


       transaction_id             ip  device_id  device_type  tran_code   mcc  \
0             1157921    91.107.0.90      982.0            4         30  7103   
1             1595205   37.0.127.150     1033.0            4         30  2031   
2             1656884  185.233.0.240       96.0            0         25  4629   
3             1673909    91.107.0.90      982.0            4         30  4629   
4             1689203   188.43.0.135     1413.0            0         10  5840   
...               ...            ...        ...          ...        ...   ...   
53074         9919604    91.122.0.15      431.0            0         25  3193   
53075         9902369    91.122.0.15      431.0            0         10  7071   
53076         9939711     2.94.0.240     1042.0            0         10  3686   
53077         9876313   188.0.162.75      494.0            0         25  7602   
53078         9791767    46.0.43.105     1489.0            0         20  9729   

       client_id  card_type

In [18]:
data_sorted['expiration_date'] = pd.to_datetime(data_sorted['expiration_date'], errors='coerce')

data_sorted['datetime'] = pd.to_datetime(data_sorted['datetime'], errors='coerce')
# 1. Извлекаем день недели
data_sorted['day_of_week'] = data_sorted['datetime'].dt.dayofweek

# 2. Извлекаем час дня
data_sorted['hour_of_day'] = data_sorted['datetime'].dt.hour

# 3. Извлекаем месяц
data_sorted['month'] = data_sorted['datetime'].dt.month

# 4. Рассчитываем разницу во времени (в секундах) между транзакциями одного клиента
data_sorted['time_diff'] = data_sorted.groupby('client_id')['datetime'].diff().dt.total_seconds()


In [19]:
# Печать отсортированного датасета
print(data_sorted)

       transaction_id             ip  device_id  device_type  tran_code   mcc  \
0             1157921    91.107.0.90      982.0            4         30  7103   
1             1595205   37.0.127.150     1033.0            4         30  2031   
2             1656884  185.233.0.240       96.0            0         25  4629   
3             1673909    91.107.0.90      982.0            4         30  4629   
4             1689203   188.43.0.135     1413.0            0         10  5840   
...               ...            ...        ...          ...        ...   ...   
53074         9919604    91.122.0.15      431.0            0         25  3193   
53075         9902369    91.122.0.15      431.0            0         10  7071   
53076         9939711     2.94.0.240     1042.0            0         10  3686   
53077         9876313   188.0.162.75      494.0            0         25  7602   
53078         9791767    46.0.43.105     1489.0            0         20  9729   

       client_id  card_type

In [30]:
# Удаляем последний столбец
data_sorted = data_sorted.drop(columns=[data_sorted.columns[-1]])

# Проверим, что столбец удален
print(data_sorted.head())
data_sorted.to_csv('modified_data_encode_sorted_datatime.csv', index=False)

   transaction_id             ip  device_id  device_type  tran_code   mcc  \
0         1107483  109.126.0.150      376.0            0         10  9534   
1         1132931    151.0.38.90      394.0            0        451  6735   
2         1297102  194.0.144.180      175.0            3        451  7349   
3         1338602    195.58.0.15     1425.0            0         10  9618   
4         1338948    188.0.0.105     1213.0            2         20  2440   

   client_id  card_type  pin_inc_count  card_status expiration_date  \
0  101648380          0              0            0      2030-07-04   
1  101648380          0              0            0      2030-07-04   
2  101648380          0              0            0      2030-07-04   
3  101648380          0              0            0      2030-07-04   
4  101648380          0              0            0      2030-07-04   

             datetime     sum  oper_type    balance  ip_city  day_of_week  \
0 2022-02-09 17:50:08  506.24    

In [21]:
data_sorted.dtypes

transaction_id              int64
ip                         object
device_id                 float64
device_type                 int64
tran_code                   int64
mcc                         int64
client_id                   int64
card_type                   int64
pin_inc_count               int64
card_status                 int64
expiration_date    datetime64[ns]
datetime           datetime64[ns]
sum                       float64
oper_type                   int64
balance                   float64
ip_city                     int64
day_of_week                 int32
hour_of_day                 int32
month                       int32
time_diff                 float64
dtype: object

In [24]:
# Преобразование столбца 'device_id' из float в int
data_sorted['device_id'] = data_sorted['device_id'].astype(int)

# Округление столбцов 'balance' и 'sum' до 2 знаков после запятой
data_sorted['balance'] = data_sorted['balance'].round(2)
data_sorted['sum'] = data_sorted['sum'].round(2)
# Удаляем последний столбец
data_sorted = data_sorted.drop(data_sorted.columns[-1], axis=1)
# Выводим результат
print(data_sorted)
data_sorted.to_csv('modified_data_encode_sorted_datatime.csv', index=False)


       transaction_id             ip  device_id  device_type  tran_code   mcc  \
0             1157921    91.107.0.90        982            4         30  7103   
1             1595205   37.0.127.150       1033            4         30  2031   
2             1656884  185.233.0.240         96            0         25  4629   
3             1673909    91.107.0.90        982            4         30  4629   
4             1689203   188.43.0.135       1413            0         10  5840   
...               ...            ...        ...          ...        ...   ...   
53074         9919604    91.122.0.15        431            0         25  3193   
53075         9902369    91.122.0.15        431            0         10  7071   
53076         9939711     2.94.0.240       1042            0         10  3686   
53077         9876313   188.0.162.75        494            0         25  7602   
53078         9791767    46.0.43.105       1489            0         20  9729   

       client_id  card_type